# Librerias

In [1]:
import numpy as np
import math
import sys
import random

## Simulation parameters

In [2]:
Number_Reps =10
Initial_SimTime =30
scvX = 0.9
rho1X = 0

## Rate Function

In [77]:
def Rate_function(n_reps,initial_simtime,scvX,rho1X):
    TwoMomsOrder=[]
    ImpThirdMoms=0
    IntNote = 10
    StepSize = 0.01
    MaxMult = 10

    if initial_simtime/StepSize+1 > 15000:
        StepSize = StepSize*10
    if IntNote < n_reps/2:
        IntNote = np.floor(n_reps/2)
    elif IntNote < n_reps/200:
        IntNote = np.floor(n_reps/200) # o int()
    NumberofSteps = (initial_simtime/StepSize)+1
    
    MeanSvTime = 1
    TrueVarMoms = scvX*(MeanSvTime**2) # Acá obtengo la varianza
    TrueSecMoms = TrueVarMoms+pow(MeanSvTime,2) # y acá el segundo momento de la variaza(Var[X]+E[X]^2)
    Coeffvar = np.sqrt(scvX) # Determino el coeficiente de variación
    
    if scvX == 1:
        # exponential
        TwoMomsOrder = 1
        ImpThirdMoms = 6 # E[X^3]=3!*(1/lambda)^3, con lambda=1 da 6
    elif scvX<1:
        #MECO (Mixture of Erlang of Common Order)
        k=1
        while 1/k>scvX:
            k+=1
        #Parametros    
        MEConOrder = k
        MEConAlpha = (1/(1+scvX))*(k*scvX - np.sqrt(k*(1+scvX)-(k**2)*scvX))
        MEConRate = (k-MEConAlpha)/MeanSvTime
        SteadyVector = []
        SteadyCumVector = []
        for i in range(2*k-1):
            SteadyVector.append(MEConAlpha/(k-1+MEConAlpha))
        for i in range(k+1,2*k-1):
            SteadyVector.append((1-MEConAlpha)/(k-1+MEConAlpha))
        
        TwoMomsOrder = 2*MEConOrder-1
        TMat = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        TMatInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        TMatSqInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))        
        TMatCuInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))

        
        ZetaVec = [0 for x in range(TwoMomsOrder)]
        ZetaVec[0] = 1-MEConAlpha
        ZetaVec[MEConOrder] = MEConAlpha
        for i in range(MEConOrder):
            for j in range(MEConOrder):
                TMatInv[i][j] = -1/MEConRate 
        for i in range(MEConOrder+1,2*MEConOrder-1):
            for j in range(i,2*MEConOrder-1):
                TMatInv[i][j] = -1/MEConRate         



    else:
        # Hyperexponencial
        TwoMomsOrder = 2
        TMat = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        TMatInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        TMatCuInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        TMatSqInv = np.zeros(shape=(TwoMomsOrder,TwoMomsOrder))
        ZetaVec = [0 for x in range(TwoMomsOrder)]
        SteadyVector = []
        for i in range(TwoMomsOrder):
            SteadyVector.append(1/TwoMomsOrder)
            
        h2bAlpha = 0.5*(1+np.sqrt(1-2/(scvX+1)))
        h2bRate = 2*h2bAlpha/MeanSvTime
        
        TMat[0,0] = -h2bRate
        TMat[1,1] = -h2bRate*(1-h2bAlpha)/h2bAlpha
        ZetaVec[0] = h2bAlpha
        ZetaVec[1] = 1-h2bAlpha
        for i in range(TwoMomsOrder):
            TMatInv[i][i] = 1/TMat[i][i]
            
    if scvX>1 or scvX<1:
        #Se calcula el tercer momento :s
        for i in range(TwoMomsOrder):
            for j in range(TwoMomsOrder):
                for k in range(TwoMomsOrder):
                    TMatSqInv[i][j] = TMatSqInv[i][j]+TMatInv[i][k]*TMatInv[k][j]
                    
        for i in range(TwoMomsOrder):
            for j in range(TwoMomsOrder):
                for k in range(TwoMomsOrder):
                    TMatCuInv[i][j] = TMatCuInv[i][j]+TMatInv[i][k]*TMatSqInv[k][j]
        
        for i in range(TwoMomsOrder):
            for j in range(TwoMomsOrder):
                ImpThirdMoms = ImpThirdMoms-6*ZetaVec[i]*TMatCuInv[i][j]
    
    #Se busca el markov MECO que calze
    SkewX = (ImpThirdMoms-3*MeanSvTime*TrueSecMoms+2*pow(MeanSvTime,3))/(pow(TrueVarMoms,1.5))
    MajPhType = 5 #Este es el parametro que dijo el profe

    if Coeffvar>0 and SkewX>= (Coeffvar- (1/Coeffvar)):
        MinN1 = 1/scvX
        MinN2 = (-SkewX+1/Coeffvar**3+1/Coeffvar+2*Coeffvar)/(SkewX- (Coeffvar-1/Coeffvar))
        
        #chequea factibilidad pero no entiendo cómo?
        if MinN1>MinN2:
            k = int(MinN1)+1
        else:
            k = int(MinN2)+1 
    else:
        print("Bad MECO")
        sys.exit(0)
        

    NeedBiggerOrder = True
    while NeedBiggerOrder == True:
        mecoM1 = MeanSvTime
        mecoM2 = TrueSecMoms
        mecoM3 = ImpThirdMoms
        
        mecoX = mecoM1*mecoM3-((k+2)/(k+1))*(mecoM2**2)
        mecoY = mecoM2-((k+1)/k)*(mecoM1**2)
        mecoA = k*(k+2)*mecoM1*mecoY
        mecoB = -(k*mecoX+k*((k+2)/(k+1))*(mecoY**2)+(k+2)*mecoY*(mecoM1**2))
        mecoC = mecoM1*mecoX

        mecoRoot1 = (-mecoB + math.sqrt((mecoB**2) - 4 * mecoA * mecoC)) / (2 * mecoA)
        mecoRoot2 = (-mecoB - math.sqrt((mecoB**2) - 4 * mecoA * mecoC)) / (2 * mecoA)
        
        MECOOrder = k
        MECORate1 = 1/mecoRoot1
        MECORate2 = 1/mecoRoot2
        MECOAlpha = ((mecoM1/MECOOrder)-mecoRoot2)/(mecoRoot1-mecoRoot2)
        
        # A continuacion trabaja con el MArkovMeco
        tempMMECOCov = TrueVarMoms*rho1X
        MMECOAlpha21 = MECOAlpha-tempMMECOCov/((1-MECOAlpha)*((MECOOrder*mecoRoot1-MECOOrder*mecoRoot2)**2))
        MMECOAlpha12 = MMECOAlpha21*(1-MECOAlpha)/MECOAlpha
        MMECORate1 = MECORate1
        MMECORate2 = MECORate2
        
        #Chequea factibilidad
        if MMECOAlpha21<0 or MMECOAlpha21>1 or MMECOAlpha12<0 or MMECOAlpha12>1:
            k += 1
            if k>4000:
                print("No se pueden unir los momentos,chequear un valor factible para rho1....")
                sys.exit(0)
                
        else:
            NeedBiggerOrder = False
            MMECOOrder = MECOOrder         
    SteadyVector = []
    
    for i in range(MMECOOrder):
        SteadyVector.append((MECORate2*MMECOAlpha21)/(MMECOOrder*(MECORate1*MMECOAlpha12+MECORate2*MMECOAlpha21)))
    
    for i in range(MMECOOrder+1,2*MMECOOrder):
        SteadyVector.append((MECORate1*MMECOAlpha12)/(MMECOOrder*(MECORate1*MMECOAlpha12+MECORate2))) 
    
    MMECOCInf = scvX*(1+2*MECOAlpha*rho1X/MMECOAlpha21)
    
    SteadyCumVector = [SteadyVector[0]]
    l = SteadyCumVector[0]
    for i in range(1,len(SteadyVector)):
        SteadyCumVector.append(SteadyVector[i]+SteadyCumVector[i-1])
#         print(SteadyCumVector)
    return (ImpThirdMoms,SteadyCumVector,MMECOOrder,MMECORate1,MMECORate2)

In [79]:
SteadyCumVector=[]
ImpThirdMoms,SteadyCumVector,MMECOOrder,MMECORate1,MMECORate2=Rate_function(n_reps=100,initial_simtime=10,scvX=1.5,rho1X=0.5)
print(ImpThirdMoms)
print(SteadyCumVector)
print(MMECOOrder)
print(MMECORate1)
print(MMECORate2)

11.249999999999996
[0.17383885804853247, 0.34767771609706494, 0.5215165741455974, 0.5287852559708933, 0.5360539377961893]
3
0.9783727202505
5.2034454615676875


## First Potential Arrival

In [38]:
def ErlangOneStep(n,Beta):
    Prod=0.01
    for i in range(n): 
        U = random.uniform(0, 1) 
        Prod = Prod * (1 - U)
#         print(Prod)
    ErlangOneStep = -Beta * math.log(Prod)
    return(ErlangOneStep)

In [88]:
def first_potential_arrival(MajPhType,MMECOOrder,MMECORate1,MMECORate2):
    if MajPhType == 5:
        WhichBasePh = random.uniform(0, 1) 
        R = 0
        while WhichBasePh < SteadyCumVector[R]:
            R +=1
        CurPhase = R
        if CurPhase < MMECOOrder +1:
            FirstArrTime = ErlangOneStep(n=MMECOOrder+1-CurPhase,Beta=(1/MMECORate1))
            PrevGenErlang = 1
        else:
            FirstArrTime = ErlangOneStep(n=MMECOOrder+1-CurPhase,Beta=1/MMECORate2)
            PrevGenErlang = 2
    return(FirstArrTime)                      

In [89]:
print(MMECOOrder)
FirstArrTime=first_potential_arrival(MajPhType=5,MMECOOrder=MMECOOrder,MMECORate1=MMECORate1,MMECORate2=MMECORate2)

3


# MakeArrival

In [90]:
def MakeArrival(MajPhType):
     if MajPhType == 5:
            WhichBasePh =random.uniform(0, 1)
            if PrevGenErlang == 1:  
                if WhichBasePh < (1 - MMECOAlpha12): 
                    NextArrTime = ErlangOneStep(MMECOOrder, 1 / MMECORate1, ExponStream) 
                    PrevGenErlang = 1
                else:
                    NextArrTime = ErlangOneStep(MMECOOrder, 1 / MMECORate2, ExponStream) 
                    PrevGenErlang = 2
            else:
                if WhichBasePh < MMECOAlpha21: 
                    NextArrTime = ErlangOneStep(MMECOOrder, 1 / MMECORate1, ExponStream) 
                    PrevGenErlang = 1
                else:
                    NextArrTime = ErlangOneStep(MMECOOrder, 1 / MMECORate2, ExponStream) 
                    PrevGenErlang = 2


In [91]:
MakeArrival(MajPhType=5)

UnboundLocalError: local variable 'PrevGenErlang' referenced before assignment

# Anexos que quizas use

In [59]:
# MODLUS = 2147483647 
# MULT1 = 24112 
# MULT2 = 26143
# zrng=[0 for i in range(100)]
# zrng[0] = 1973272912
# zrng[1] = 281629770
# zrng[2] = 20006270 
# zrng[3] = 1280689831
# zrng[4] = 2096730329 
# zrng[5] = 1933576050 
# zrng[6] = 913566091 
# zrng[7] = 246780520 
# zrng[8] = 1363774876 
# zrng[9] = 604901985 
# zrng[10] = 1511192140 
# zrng[11] = 1259851944 
# zrng[12] = 824064364 
# zrng[13] = 150493284 
# zrng[14] = 242708531 
# zrng[15] = 75253171 
# zrng[16] = 1964472944 
# zrng[17] = 1202299975 
# zrng[18] = 233217322 
# zrng[19] = 1911216000 
# zrng[20] = 726370533 
# zrng[21] = 403498145 
# zrng[22] = 993232223 
# zrng[23] = 1103205531 
# zrng[24] = 762430696 
# zrng[25] = 1922803170 
# zrng[26] = 1385516923 
# zrng[27] = 76271663 
# zrng[28] = 413682397 
# zrng[29] = 726466604 
# zrng[30] = 336157058 
# zrng[31] = 1432650381 
# zrng[32] = 1120463904 
# zrng[33] = 595778810 
# zrng[34] = 877722890 
# zrng[35] = 1046574445 
# zrng[36] = 68911991 
# zrng[37] = 2088367019
# zrng[38] = 748545416
# zrng[39] = 622401386 
# zrng[40] = 2122378830 
# zrng[41] = 640690903 
# zrng[42] = 1774806513 
# zrng[43] = 2132545692 
# zrng[44] = 2079249579 
# zrng[45] = 78130110 
# zrng[46] = 852776735 
# zrng[47] = 1187867272 
# zrng[48] = 1351423507 
# zrng[49] = 1645973084 
# zrng[50] = 1997049139 
# zrng[51] = 922510944 
# zrng[52] = 2045512870 
# zrng[53] = 898585771 
# zrng[54] = 243649545 
# zrng[55] = 1004818771
# zrng[56] = 773686062 
# zrng[57] = 403188473 
# zrng[58] = 372279877 
# zrng[59] = 1901633463 
# zrng[60] = 498067494 
# zrng[61] = 2087759558 
# zrng[62] = 493157915 
# zrng[63] = 597104727 
# zrng[64] = 1530940798 
# zrng[65] = 1814496276 
# zrng[66] = 536444882 
# zrng[67] = 1663153658 
# zrng[68] = 855503735 
# zrng[69] = 67784357 
# zrng[70] = 1432404475 
# zrng[71] = 619691088 
# zrng[72] = 119025595 
# zrng[73] = 880802310 
# zrng[74] = 176192644 
# zrng[75] = 1116780070 
# zrng[76] = 277854671 
# zrng[77] = 1366580350 
# zrng[78] = 1142483975 
# zrng[79] = 2026948561 
# zrng[80] = 1053920743 
# zrng[81] = 786262391 
# zrng[82] = 1792203830 
# zrng[83] = 1494667770 
# zrng[84] = 1923011392 
# zrng[85] = 1433700034 
# zrng[86] = 1244184613 
# zrng[87] = 1147297105 
# zrng[88] = 539712780 
# zrng[89] = 1545929719 
# zrng[90] = 190641742 
# zrng[91] = 1645390429 
# zrng[92] = 264907697 
# zrng[93] = 620389253 
# zrng[94] = 1502074852 
# zrng[95] = 927711160 
# zrng[96] = 364849192 
# zrng[97] = 2049576050 
# zrng[98] = 638580085 
# zrng[99] = 547070247